# Web Scrapping with Selenium
### Import Libraries

We'll first import all relevant libraries that we will require to access a javascript driven website's HTML and extract information from the same.

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd

### Set up chrome driver
Set up chrome driver in headless mode and disable logging of Devtools messages with help of selenium library.

In [2]:
service = Service('path_to_chromedriver')  
chrome_options = Options()
chrome_options.add_argument('--headless')#  # Run Chrome in headless mode, remove this line if you want to see the browser window
chrome_options.add_argument('--disable-gpu')  # Disable GPU acceleration, as it may cause issues in headless mode
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])  # Disable logging of DevTools messages
driver = webdriver.Chrome(service = service,options=chrome_options)

### Fetching Table headers of Listed Stocks Data from the First Pages of Website.
We want to scrap the Listed stocks data from : https://www.screener.in/screens/29729/top-1000-stocks/?page={1}

In [3]:
url_pattern = f"https://www.screener.in/screens/29729/top-1000-stocks/?page={1}"
driver.get(url_pattern)
table = driver.find_element(By.CLASS_NAME, "data-table")

# Extract the table headers
headers = ([th.text.strip() for th in table.find_elements(By.TAG_NAME, "th")])[0:12]
headers

['S.No.',
 'Name',
 'CMP Rs.',
 'P/E',
 'Mar Cap Rs.Cr.',
 'Div Yld %',
 'NP Qtr Rs.Cr.',
 'Qtr Profit Var %',
 'Sales Qtr Rs.Cr.',
 'Qtr Sales Var %',
 'ROCE %',
 'PAT Ann Rs.Cr.']

### Fetching complete table data of listed stocks from all pages of website.

In [4]:
# Empty list to store all the stock data
stock_data = []

# Iterate through all the pages
for page in range(1, 51):
    # Construct the URL for the current page
    url = f"https://www.screener.in/screens/29729/top-1000-stocks/?page={page}"

    # Load the page
    driver.get(url)

    # Find the table containing the stock data
    table = driver.find_element(By.CLASS_NAME, "data-table")
    
    # Extract the table rows
    rows = table.find_elements(By.TAG_NAME, "tr")

    # Extract the data from each row
    for row in rows[1:-1]:
        data = [td.text.strip() for td in row.find_elements(By.TAG_NAME, "td")]
        if len(data)==0:
            pass
        else:
            stock_data.append(data)

# Quit the driver
driver.quit()

### Importing the Data into Pandas DataFrame and Extracting it To CSV file

In [5]:
# Create a pandas DataFrame from the stock data
df = pd.DataFrame(stock_data, columns=headers)

# Display the DataFrame
# print(df)
df.to_csv("All_Stocks_Row_Data.csv",index=False)

In [6]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1230 entries, 0 to 1229
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   S.No.             1230 non-null   object
 1   Name              1230 non-null   object
 2   CMP Rs.           1230 non-null   object
 3   P/E               1230 non-null   object
 4   Mar Cap Rs.Cr.    1230 non-null   object
 5   Div Yld %         1230 non-null   object
 6   NP Qtr Rs.Cr.     1230 non-null   object
 7   Qtr Profit Var %  1230 non-null   object
 8   Sales Qtr Rs.Cr.  1230 non-null   object
 9   Qtr Sales Var %   1230 non-null   object
 10  ROCE %            1230 non-null   object
 11  PAT Ann Rs.Cr.    1230 non-null   object
dtypes: object(12)
memory usage: 115.4+ KB
None


## Data Cleaning 
Most of case scraped dataset is not available in the most useful format and must be cleaned before any analysis could be performed.

In this case, data type of all colums are object, we need to change in suitable type.<br>
For that we read stored csv file in a new pandas dataframe, that changes columns data type.<br>

In [7]:
import pandas as pd
ndf = pd.read_csv("All_Stocks_Row_Data.csv",na_values=["not available","n.a."],index_col=[0])

In [8]:
ndf.loc[[1138,1147,1187]]

,Name,CMP Rs.,P/E,Mar Cap Rs.Cr.,Div Yld %,NP Qtr Rs.Cr.,Qtr Profit Var %,Sales Qtr Rs.Cr.,Qtr Sales Var %,ROCE %,PAT Ann Rs.Cr.
S.No.,,,,,,,,,,,
1138.0,The Hi-Tech Gear,353.40,28.70,663.33,0.42,16.64,48.04,286.55,-0.72,8.32,23.11
1147.0,Marine Electric.,50.65,32.79,639.79,0.00,6.23,28.72,129.68,69.78,17.15,19.51
1187.0,Supershakti Met.,500.00,17.31,576.26,0.20,10.44,NaN,370.53,NaN,24.20,33.30


In [9]:
ndf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1230 entries, 1.0 to 1230.0
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              1230 non-null   object 
 1   CMP Rs.           1230 non-null   float64
 2   P/E               1229 non-null   float64
 3   Mar Cap Rs.Cr.    1230 non-null   float64
 4   Div Yld %         1224 non-null   float64
 5   NP Qtr Rs.Cr.     1230 non-null   float64
 6   Qtr Profit Var %  1216 non-null   float64
 7   Sales Qtr Rs.Cr.  1230 non-null   float64
 8   Qtr Sales Var %   1214 non-null   float64
 9   ROCE %            1227 non-null   float64
 10  PAT Ann Rs.Cr.    1230 non-null   float64
dtypes: float64(10), object(1)
memory usage: 147.6+ KB


### Create a new dataframe 'new_df' and take selectd columns from 'ndf' dataframe

In [10]:
new_df = ndf[["Name","CMP Rs.","P/E","Div Yld %","ROCE %"]]

In [11]:
new_df

,Name,CMP Rs.,P/E,Div Yld %,ROCE %
S.No.,,,,,
1.0,Reliance Industr,2481.95,25.17,0.32,10.14
2.0,TCS,3209.30,27.88,1.50,59.11
3.0,HDFC Bank,1610.60,19.57,1.18,6.85
4.0,ICICI Bank,938.05,19.27,0.53,6.82
5.0,Hind. Unilever,2636.20,60.91,1.48,26.61
...,...,...,...,...,...
1226.0,Kamat Hotels,206.95,6.62,0.00,25.66
1227.0,N R Agarwal Inds,297.75,5.10,0.00,21.39
1228.0,Andhra Petrochem,59.56,24.80,2.52,6.40


### Export data from "new_df" dataframe to a "clean_Data.csv" file.

In [12]:
new_df.to_csv("Clean_Data.csv")